In [ ]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import copy
import math

import numpy as np
import torch
import socialforce

(corridor)=
# Corridor

We want to qualitatively study the impact of non-standard Social Force potentials.

In [ ]:
def initial_state_people(n):
    _ = torch.manual_seed(42)

    # first n people are go the right, second n people go left
    state = torch.zeros((n * 2, 6))

    # positions
    state[:n, 0:2] = ((torch.rand((n, 2)) - 0.5) * 2.0) * torch.tensor([25.0, 4.5])
    state[n:, 0:2] = ((torch.rand((n, 2)) - 0.5) * 2.0) * torch.tensor([25.0, 4.5])

    # velocity
    state[:n, 2:4] = torch.normal(torch.full((n, 1), 1.34), 0.26)
    state[n:, 2:4] = torch.normal(torch.full((n, 1), -1.34), 0.26)

    # x destination
    state[:n, 4] = 100.0
    state[n:, 4] = -100.0

    return state

upper_wall = torch.stack([torch.linspace(-30, 30, 600), torch.full((600,), 5)], -1)
lower_wall = torch.stack([torch.linspace(-30, 30, 600), torch.full((600,), -5)], -1)
ped_space = socialforce.PedSpacePotential([upper_wall, lower_wall])


## Rereference Potential

Standard SF:

In [ ]:
ped_ped = socialforce.potentials.PedPedPotential()
initial_state = initial_state_people(30)
simulator = socialforce.Simulator(ped_ped=ped_ped, ped_space=ped_space)
simulator.integrator = socialforce.simulator.PeriodicBoundary(simulator.integrator, x_boundary=[-30.0, 30.0])

with torch.no_grad():
    states = simulator.run(initial_state, 500)

In [ ]:
# HIDE CODE
with socialforce.show.track_canvas(ncols=3, figsize=(12, 4)) as (ax1, ax2, ax3):
    socialforce.show.states(ax1, states[0:1], monochrome=True)
    ax1.set_xlim(-30, 30)

    socialforce.show.states(ax2, states[50:51], monochrome=True)
    ax2.set_xlim(-30, 30)

    socialforce.show.states(ax3, states[499:500], monochrome=True)
    ax3.set_xlim(-30, 30)